In [1]:
import pandas as pd
from datetime import datetime

## 1.- Cargar el archivo de datos y eliminar duplicados
reservas = pd.read_excel("clientes.xlsx").drop_duplicates(subset='id_booking')

Bookings

In [2]:
## 2.- Filtrar las reservas según su estado
reservas = reservas[~reservas['status_booking'].isin(['Cancelado', 'Reemplazado', 'Pendiente', 'En revisión'])] 

In [3]:
## 3.- Convertir zla fecha de operación de la reserva a formato año-mes
reservas['date_operation'] = pd.to_datetime(reservas['date_booking_operation']).dt.strftime('%Y-%m') 

## 4.- Reordenar las columnas y renombrar una columna
reservas = reservas.rename(columns={'date_operation': 'date_booking'}) 

In [4]:
# 5.- Agrupar las reservas por propietario y agregar los datos como listas
df_aslist = reservas.groupby(['id_owner']).agg(lambda x: list(x)).reset_index() 
# Se agrupan las reservas por id_owner y se agregan los datos como listas. El resultado se almacena en un nuevo DataFrame

In [5]:
## 6.- Crear un DataFrame para almacenar los usuarios activos mensuales (MAU)
months = df_aslist['date_booking'].explode().unique() 
months = sorted(months)

In [6]:
# Se crea un DataFrame vacío llamado MAU con una columna para cada mes en el DataFrame df_aslist. 
MAU = pd.DataFrame(index=['DMAU', 'MAU', 'DMRU', 'MRU', 'MNU', 'DMRSU', 'MRSU'], columns=months) 

In [7]:
## 7.- Calcular los usuarios activos mensuales (MAU)
for element in months: 
    active = df_aslist[df_aslist['date_booking'].apply(lambda x: element in x)]['id_owner'].tolist() 
    MAU.at['DMAU', element] = active 
    MAU.at['MAU', element] = len(active) 


In [8]:
## 8.- Calcular los meses de usuarios retenidos(MRU) 
lista_retained = [[]] 
total_retained = [0] 
for counter in range(1, MAU.shape[1]): 
    retained = list(set(MAU.loc['DMAU'][counter]) & set(MAU.loc['DMAU'][counter-1])) 
    lista_retained.append(retained) 
    total_retained.append(len(retained)) 
MAU.loc['DMRU'] = lista_retained 
MAU.loc['MRU'] = total_retained 
MAU.loc['MNU'] = MAU.loc['MAU'] - MAU.loc['MRU'] 

In [9]:
## 9.- Calcular los meses de usuarios resucitados(MRSU)  
hist_user_real = [[], []] 
hist_user_count = [0, 0] 
hist_users = set() 
hist_users.add(MAU.loc['DMAU'][0][0]) 
for counter in range(1, MAU.shape[1]): 
    if counter == 1: 
        hist_users = hist_users | set(MAU.loc['DMAU'][counter]) 
    else: 
        NR_monthly_users = set(MAU.loc['DMAU'][counter]) - set(MAU.loc['DMRU'][counter]) 
        New_monthly_users = NR_monthly_users & hist_users 
        hist_user_real.append(list(New_monthly_users)) 
        hist_user_count.append(len(list(New_monthly_users))) 
        hist_users = hist_users | set(MAU.loc['DMAU'][counter]) 
MAU.loc['DMRSU'] = hist_user_real 
MAU.loc['MRSU'] = hist_user_count 
MAU.loc['MNU'] = MAU.loc['MNU'] - MAU.loc['MRSU'] 

In [10]:
# Transpose the DataFrame and rename the columns 
Final_MAU = MAU.T 
columns = ['DMAU', 'MAU', 'DMRU', 'MRU', 'MNU', 'DMRSU', 'MRSU'] 
Final_MAU.columns = columns 

In [11]:
DMNU = [] #Lista de usuarios nuevos
for index, row in Final_MAU.iterrows():
    DMNU.append( list( set(row['DMAU']) - (set(row['DMRU']) | set(row['DMRSU']) ) ) )

lista_nombres = Final_MAU.columns
LN=list(lista_nombres)
change_index=LN.index('MRU')
change_index
LN.insert(change_index + 1,'DMNU')
Final_MAU['DMNU'] = DMNU
Final_MAU=Final_MAU[LN]

In [12]:
row_names = list(Final_MAU.index)
churned = [[]]
churned_q = [0]
for counter in range(1,len(row_names)):
    churned_users = set(Final_MAU.iloc[counter-1][0]) -  set(Final_MAU.iloc[counter][0])
    churned.append(list(churned_users))
    churned_q.append(-len(churned_users))

Final_MAU['DMCU'] = churned
Final_MAU['MCU'] = churned_q

Revenue

In [13]:
resurr_income = []
resurr_reser = []

new_income = []
new_reser = []

retained_income =[]
retained_reser = []

expansion_income = [0]
expansion_reser = [0]

contraction_income = [0]
contraction_reser = [0]

churned_income = [0]
churned_reser = [0]

aux_row = None
for row in row_names:
        if row == '2020-07':
            row_data = Final_MAU.loc[row]


            resurr_owners = row_data['DMRSU']
            new_owners = row_data['DMNU']
            retained_owners = row_data['DMRU']

            res_income = 0
            n_income = 0
            r_income = 0 

            res_reser = 0
            n_reser = 0
            r_reser = 0

            if row < '2022-01':
                for user in resurr_owners:
                    aux = reservas[reservas['id_owner']==user]
                    res_income = res_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    res_reser = res_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in new_owners:
                    aux = reservas[reservas['id_owner']==user]
                    n_income = n_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    n_reser = n_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in retained_owners:
                    aux = reservas[reservas['id_owner']==user]
                    r_income = r_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    r_reser = r_reser + aux[aux['date_booking']==row]['id_owner'].count()
    


            resurr_income.append(res_income)
            resurr_reser.append(res_reser)

            new_income.append(n_income)
            new_reser.append(n_reser)

            retained_income.append(r_income)
            retained_reser.append(r_reser)

            aux_row = row

        else:
            row_data = Final_MAU.loc[row]
            prev_row_data = Final_MAU.loc[aux_row]

            resurr_owners = row_data['DMRSU']
            new_owners = row_data['DMNU']
            retained_owners = row_data['DMRU']
            churned_owners = row_data['DMCU']

            res_income = 0
            n_income = 0
            r_income = 0 
            exp_income = 0
            cont_income = 0
            ch_income = 0

            res_reser = 0
            n_reser = 0
            r_reser = 0
            exp_reser = 0
            cont_reser = 0
            ch_reser = 0

            if row < '2022-01':
                for user in resurr_owners:
                    aux = reservas[reservas['id_owner']==user]
                    res_income = res_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    res_reser = res_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in new_owners:
                    aux = reservas[reservas['id_owner']==user]
                    n_income = n_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    n_reser = n_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in churned_owners:
                    aux = reservas[reservas['id_owner'] == user]
                    ch_income = ch_income + aux[aux['date_booking'] == aux_row]['value_total_reservation'].sum()*0.15
                    ch_reser = ch_reser + aux[aux['date_booking']==aux_row]['id_owner'].count()

                for user in retained_owners:
                    aux = reservas[reservas['id_owner']==user]
                    user_booking = aux[aux['date_booking']==row]['id_owner'].count()
                    user_income = aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.15
                    user_past_booking = aux[aux['date_booking']==aux_row]['id_owner'].count()
                    user_past_income = aux[aux['date_booking']==aux_row]['value_total_reservation'].sum()*0.15 
                    crit = user_booking - user_past_booking
                    x_income = user_income - user_past_income
                    if crit > 0:
                        r_reser = r_reser + user_past_booking
                        exp_reser = exp_reser + crit
                    elif crit == 0:
                        r_reser = r_reser + user_booking
                    else:
                        cont_reser = cont_reser + crit
                        r_reser = r_reser + user_booking
                    if x_income >0:
                        r_income = r_income + user_past_income
                        exp_income = exp_income + x_income
                    elif x_income == 0:
                        r_income = r_income + user_income
                    else:
                        cont_income = cont_income + x_income
                        r_income = r_income + user_income
            else :
                for user in resurr_owners:
                    aux = reservas[reservas['id_owner']==user]
                    res_income = res_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.20
                    res_reser = res_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in new_owners:
                    aux = reservas[reservas['id_owner']==user]
                    n_income = n_income + aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.20
                    n_reser = n_reser + aux[aux['date_booking']==row]['id_owner'].count()

                for user in churned_owners:
                    aux = reservas[reservas['id_owner'] == user]
                    ch_income = ch_income + aux[aux['date_booking'] == aux_row]['value_total_reservation'].sum()*0.20
                    ch_reser = ch_reser + aux[aux['date_booking']==aux_row]['id_owner'].count()

                for user in retained_owners:
                    aux = reservas[reservas['id_owner']==user]
                    user_booking = aux[aux['date_booking']==row]['id_owner'].count()
                    user_income = aux[aux['date_booking']==row]['value_total_reservation'].sum()*0.20
                    user_past_booking = aux[aux['date_booking']==aux_row]['id_owner'].count()
                    user_past_income = aux[aux['date_booking']==aux_row]['value_total_reservation'].sum()*0.20
                    crit = user_booking - user_past_booking
                    x_income = user_income - user_past_income
                    if crit > 0:
                        r_reser = r_reser + user_past_booking
                        exp_reser = exp_reser + crit
                    elif crit == 0:
                        r_reser = r_reser + user_booking
                    else:
                        cont_reser = cont_reser + crit
                        r_reser = r_reser + user_booking
                    if x_income >0:
                        r_income = r_income + user_past_income
                        exp_income = exp_income + x_income
                    elif x_income == 0:
                        r_income = r_income + user_income
                    else:
                        cont_income = cont_income + x_income
                        r_income = r_income + user_income



            resurr_income.append(res_income)
            resurr_reser.append(res_reser)

            new_income.append(n_income)
            new_reser.append(n_reser)

            retained_income.append(r_income)
            retained_reser.append(r_reser)

            expansion_income.append(exp_income)
            expansion_reser.append(exp_reser)

            contraction_income.append(cont_income)
            contraction_reser.append(cont_reser)

            churned_income.append(-ch_income)
            churned_reser.append(-ch_reser)

            aux_row = row


Final_MAU['resurr_income'] = resurr_income
Final_MAU['new_income'] = new_income
Final_MAU['retained_income'] = retained_income
Final_MAU['expansion_income'] = expansion_income
Final_MAU['contraction_income'] = contraction_income
Final_MAU['churned_income'] = churned_income


Final_MAU['resurr_bookings'] = resurr_reser
Final_MAU['new_bookings'] = new_reser
Final_MAU['retained_bookings'] = retained_reser
Final_MAU['expansion_bookings'] = expansion_reser
Final_MAU['contraction_bookings'] = contraction_reser
Final_MAU['churned_bookings']= churned_reser
Final_MAU['Total_bookings'] = Final_MAU['resurr_bookings'] + Final_MAU['new_bookings'] + Final_MAU['retained_bookings'] + Final_MAU['expansion_bookings']
Final_MAU['Total_retained'] = Final_MAU['resurr_income']  + Final_MAU['retained_income'] + Final_MAU['expansion_income']
Final_MAU['Total_income'] = Final_MAU['resurr_income'] + Final_MAU['new_income'] + Final_MAU['retained_income'] + Final_MAU['expansion_income']


In [14]:
def calculate_metrics(data_actual, data_anterior, period):
    actual_MAU = data_actual['MAU']
    anterior_MAU = data_anterior['MAU']
    actual_bookings = data_actual['Total_bookings']
    anterior_bookings = data_anterior['Total_bookings']
    actual_revenue = data_actual['resurr_income'] + data_actual['new_income'] + data_actual['retained_income'] + data_actual['expansion_income']
    anterior_revenue = data_anterior['resurr_income'] + data_anterior['new_income'] + data_anterior['retained_income'] + data_anterior['expansion_income']

    MAU_users = (((actual_MAU / anterior_MAU) ** (1/period))-1)
    monthly_bookings = (((actual_bookings / anterior_bookings) ** (1/period))-1)
    monthly_revenue = (((actual_revenue / anterior_revenue) ** (1/period))-1)

    return MAU_users, monthly_bookings, monthly_revenue
# Inicializar las listas de métricas
MAU_users_cmgr12 = [0]
monthly_bookings_cmgr12 = [0]
monthly_revenue_cmgr12 = [0]
MAU_users_cmgr6 = [0]
monthly_bookings_cmgr6 = [0]
monthly_revenue_cmgr6 = [0]
MAU_users_cmgr3 = [0]
monthly_bookings_cmgr3 = [0]
monthly_revenue_cmgr3 = [0]
MoM_user_retention = [0]
MoM_Bookings_Retention = [0]
MoM_Revenue_Retention = [0]
MAU_Quick_Ratio = [0]
Bookings_Quick_Ratio = [0]
Revenue_Quick_Ratio = [0]

# Calcular las métricas para cada período
for counter in range(1, len(row_names)):
    if counter >= 12:
        period = 12
        data_actual = Final_MAU.loc[row_names[counter]]
        data_anterior = Final_MAU.loc[row_names[counter-12]]

        MAU_users_12, monthly_bookings_12, monthly_revenue_12 = calculate_metrics(data_actual, data_anterior, period)
        MAU_users_cmgr12.append(MAU_users_12)
        monthly_bookings_cmgr12.append(monthly_bookings_12)
        monthly_revenue_cmgr12.append(monthly_revenue_12)
    else:
        period = counter
        MAU_users_cmgr12.append(0)
        monthly_bookings_cmgr12.append(0)
        monthly_revenue_cmgr12.append(0)

    if counter >= 6:
        period = 6
        data_actual = Final_MAU.loc[row_names[counter]]
        data_anterior = Final_MAU.loc[row_names[counter-6]]

        MAU_users_6, monthly_bookings_6, monthly_revenue_6 = calculate_metrics(data_actual, data_anterior, period)
        MAU_users_cmgr6.append(MAU_users_6)
        monthly_bookings_cmgr6.append(monthly_bookings_6)
        monthly_revenue_cmgr6.append(monthly_revenue_6)
    else:
        period = counter
        MAU_users_cmgr6.append(0)
        monthly_bookings_cmgr6.append(0)
        monthly_revenue_cmgr6.append(0)

    if counter >= 3:
        period = 3
        data_actual = Final_MAU.loc[row_names[counter]]
        data_anterior = Final_MAU.loc[row_names[counter-3]]

        MAU_users_3, monthly_bookings_3, monthly_revenue_3 = calculate_metrics(data_actual, data_anterior, period)
        MAU_users_cmgr3.append(MAU_users_3)
        monthly_bookings_cmgr3.append(monthly_bookings_3)
        monthly_revenue_cmgr3.append(monthly_revenue_3)
    else:
        period = counter
        MAU_users_cmgr3.append(0)
        monthly_bookings_cmgr3.append(0)
        monthly_revenue_cmgr3.append(0)

    # Métricas adicionales
    if counter >= 3:
        data_actual = Final_MAU.loc[row_names[counter]]
        data_anterior = Final_MAU.loc[row_names[counter-1]]

        actual_MoM_user = data_actual['MRU']
        anterior_MoM_user = data_anterior['MRU']+data_anterior['MRSU']+data_anterior['MNU']
        actual_MoM_Bookings = data_actual['retained_bookings']
        anterior_MoM_Bookings = data_anterior['resurr_bookings']+data_anterior['retained_bookings']+data_anterior['expansion_bookings']+data_anterior['new_bookings']
        actual_MoM_revenue = data_actual['retained_income']
        anterior_MoM_revenue = data_anterior['resurr_income']+data_anterior['retained_income']+data_anterior['expansion_income']+data_anterior['new_income']

        MoM_user_retention_1= (actual_MoM_user/anterior_MoM_user)
        MoM_Bookings_Retention_1= (actual_MoM_Bookings/anterior_MoM_Bookings)
        MoM_Revenue_Retention_1= (actual_MoM_revenue/anterior_MoM_revenue)

        MoM_user_retention.append(MoM_user_retention_1)
        MoM_Bookings_Retention.append(MoM_Bookings_Retention_1)
        MoM_Revenue_Retention.append(MoM_Revenue_Retention_1)
    else:
        MoM_user_retention_0= 0
        MoM_Bookings_Retention_0= 0
        MoM_Revenue_Retention_0= 0

        MoM_user_retention.append(MoM_user_retention_0)
        MoM_Bookings_Retention.append(MoM_Bookings_Retention_0)
        MoM_Revenue_Retention.append(MoM_Revenue_Retention_0)

    data_actual = Final_MAU.loc[row_names[counter]]

    actual_new_res = data_actual['MNU'] +data_actual['MRSU']
    actual_churned  = abs(data_actual['MCU'])
    actual_bookings_new_res_exp = data_actual['new_bookings']+data_actual['resurr_bookings']+data_actual['expansion_bookings']
    actual_churnedb = abs(data_actual['churned_bookings']+data_actual['contraction_bookings'])
    actual_revenue_new_res_exp = data_actual['new_income']+data_actual['resurr_income']+data_actual['expansion_income']
    actual_churnedr = abs(data_actual['churned_income']+data_actual['contraction_income'])

    if actual_churned > 0:
        MAU_Quick_Ratio1 = (actual_new_res)/(actual_churned)
    else:
        MAU_Quick_Ratio1 = 0

    if actual_churnedb > 0:
        Bookings_Quick_Ratio1 = (actual_bookings_new_res_exp)/(actual_churnedb)
    else:
        Bookings_Quick_Ratio1 = 0
    if actual_churnedr > 0:
        Revenue_Quick_Ratio1 = (actual_revenue_new_res_exp)/(actual_churnedr)
    else:
        Revenue_Quick_Ratio1 = 0

    MAU_Quick_Ratio.append(MAU_Quick_Ratio1)
    Bookings_Quick_Ratio.append(Bookings_Quick_Ratio1)
    Revenue_Quick_Ratio.append(Revenue_Quick_Ratio1)

# Agregar las métricas al DataFrame Final_MAU
Final_MAU['MAU_users_cmgr12'] = MAU_users_cmgr12
Final_MAU['monthly_bookings_cmgr12'] = monthly_bookings_cmgr12
Final_MAU['monthly_revenue_cmgr12'] = monthly_revenue_cmgr12
Final_MAU['MAU_users_cmgr6'] = MAU_users_cmgr6
Final_MAU['monthly_bookings_cmgr6'] = monthly_bookings_cmgr6
Final_MAU['monthly_revenue_cmgr6'] = monthly_revenue_cmgr6
Final_MAU['MAU_users_cmgr3'] = MAU_users_cmgr3
Final_MAU['monthly_bookings_cmgr3'] = monthly_bookings_cmgr3
Final_MAU['monthly_revenue_cmgr3'] = monthly_revenue_cmgr3
Final_MAU['MoM_user_retention'] = MoM_user_retention
Final_MAU['MoM_Bookings_Retention'] = MoM_Bookings_Retention
Final_MAU['MoM_Revenue_Retention'] = MoM_Revenue_Retention
Final_MAU['MAU_Quick_Ratio'] = MAU_Quick_Ratio
Final_MAU['Bookings_Quick_Ratio'] = Bookings_Quick_Ratio
Final_MAU['Revenue_Quick_Ratio'] = Revenue_Quick_Ratio

In [15]:
Final_MAU = Final_MAU.fillna(0)

In [21]:
df = Final_MAU
fechas = pd.date_range(start='2020-07', periods=len(df), freq='M')
df['Fecha'] = fechas.astype(str)
df = df.applymap(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

In [22]:
monthly_values = df.values.tolist()
monthly_values.insert(0,list(df.columns))

In [23]:
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import datetime

SERVICE_ACCOUNT_FILE = 'data-analyst.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


service = build('sheets', 'v4', credentials=creds)


In [24]:
sheet_id = "1uhXsJC--OZ-fJfvn2vi9QsMlc0bn0RTEmjrPz8CUsWU" # Id del archivo Dash de drivers
sheet = service.spreadsheets()
request = sheet.values().clear(spreadsheetId=sheet_id,
                            range="Owner").execute()
request = sheet.values().update(spreadsheetId = sheet_id,
                            range="Owner!A1", valueInputOption = "USER_ENTERED",  body = {"values": monthly_values}).execute()